In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from skimage.metrics import structural_similarity as ssim

In [ ]:
input_dir = '/Users/rathore/Documents/Logical_Rhythmm/Q3/train'
output_dir = '/Users/rathore/Documents/Logical_Rhythmm/Q3/output'

In [ ]:
def crop_borders(image, crop_percent):
    """
    Crops a specified percentage of borders from the image.
    """
    height, width = image.shape[:2]
    crop_px_height = int(height * crop_percent / 100)
    crop_px_width = int(width * crop_percent / 100)
    return image[crop_px_height:height - crop_px_height, crop_px_width:width - crop_px_width]

def convert_to_grayscale(image):
    """
    Converts an image to grayscale.
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def divide_chessboard(image, output_dir, image_id, crop_percent=0, grayscale=False):
    """
    Divides a chessboard image into 8x8 squares, applies optional grayscale and border cropping,
    and saves each square.
    """
    height, width = image.shape[:2]
    square_size = height // 8
    column_mapping = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    row_mapping = ['1', '2', '3', '4', '5', '6', '7', '8']

    for row in range(8):
        for col in range(8):
            x_start = col * square_size
            y_start = row * square_size
            x_end = x_start + square_size
            y_end = y_start + square_size
            square = image[y_start:y_end, x_start:x_end]

            if grayscale:
                square = convert_to_grayscale(square)
            if crop_percent > 0:
                square = crop_borders(square, crop_percent)

            square_filename = os.path.join(output_dir, f"{image_id}_{column_mapping[col]}{row_mapping[7 - row]}.png")
            cv2.imwrite(square_filename, square)

def display_images(output_dir, max_images=64):
    """
    Displays saved images in a grid format in alphabetical order.
    """
    image_paths = sorted(glob(os.path.join(output_dir, '*.png')))  # Sort to display in order
    rows, cols = 8, 8
    plt.figure(figsize=(15, rows * 2))

    for i, image_path in enumerate(image_paths[:max_images]):
        image = Image.open(image_path)
        plt.subplot(rows, cols, i + 1)
        plt.imshow(image, cmap='gray' if image.mode == 'L' else None)
        plt.axis('off')
        plt.title(os.path.basename(image_path), fontsize=8)

    plt.tight_layout()
    plt.savefig('foo.png', bbox_inches='tight')

# Main code
output_dir = '/Users/rathore/Documents/Logical_Rhythmm/Q3/peices_2'
input_dir = '/Users/rathore/Documents/Logical_Rhythmm/Q3/train'
os.makedirs(output_dir, exist_ok=True)

image_names = ['172384.png']
for image_name in image_names:
    image_path = os.path.join(input_dir, image_name)
    chessboard = cv2.imread(image_path)
    
    if chessboard is None:
        print(f"Failed to load image: {image_path}")
        continue

    image_id = os.path.splitext(image_name)[0]
    divide_chessboard(chessboard, output_dir, image_id, grayscale=True)

print("All images segmented, optionally converted to grayscale, cropped, and saved.")
display_images(output_dir)


In [ ]:
def load_piece_images(piece_dir):
    piece_images = {}
    for filename in glob(f"{piece_dir}/*.png"):
        piece_name = filename.split('/')[-1].split('.')[0]
        piece_images[piece_name] = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2GRAY)
    return piece_images

def extract_border(image):
    return cv2.Canny(image, 100, 200)

def compare_images(image_a, image_b):
    return ssim(image_a, image_b)

def identify_pieces(board_squares, piece_images):
    identified_board = []

    for row in board_squares:
        identified_row = []
        for square_image in row:
            square_border = extract_border(square_image)

            best_match = "blank_"
            best_score = -1 

            for piece_name, piece_image in piece_images.items():
                piece_border = extract_border(piece_image)
                score = compare_images(square_border, piece_border)
                if score > best_score:
                    best_score = score
                    best_match = piece_name

            if(best_match.split("_")[0].isdigit()):
                best_match = "blank_"
            best_match = "_".join(best_match.split("_")[:2])
            identified_row.append(best_match)
        identified_board.append(identified_row)

    return identified_board

def divide_chessboard(chessboard_image, grayscale=True):
    if grayscale:
        chessboard_image = cv2.cvtColor(chessboard_image, cv2.COLOR_BGR2GRAY)

    height, width = chessboard_image.shape
    square_size = height // 8  
    
    board_squares = []
    for row in range(8):
        row_squares = []
        for col in range(8):
            square = chessboard_image[row * square_size:(row + 1) * square_size,
                                      col * square_size:(col + 1) * square_size]
            row_squares.append(square)
        board_squares.append(row_squares)
    
    return board_squares

def save_chessboard_csv(identified_board, output_filename):
    df = pd.DataFrame(identified_board)
    df.to_csv(output_filename, index=False, header=False)

def process_chessboards_separately(test_folder, piece_dir, output_test_folder):
    if not os.path.exists(output_test_folder):
        os.makedirs(output_test_folder)

    piece_images = load_piece_images(piece_dir)

    for image_file in glob(f"{test_folder}/*.png"):
        image_name = os.path.basename(image_file)
        output_name = "_".join(image_name.split("_")[:2]) + ".csv"
        output_path = os.path.join(output_test_folder, output_name)

        chessboard_image = cv2.imread(image_file)
        if chessboard_image is None:
            print(f"Failed to load image: {image_file}")
            continue

        board_squares = divide_chessboard(chessboard_image, grayscale=True)
        identified_board = identify_pieces(board_squares, piece_images)
        save_chessboard_csv(identified_board, output_path)
        print(f"Saved identified board to {output_path}")


In [ ]:
piece_dir = "/Users/rathore/Documents/Logical_Rhythmm/Q3/peices"

In [ ]:
test_folder = '/Users/rathore/Documents/Logical_Rhythmm/Q3/test'
output_test_folder = '/Users/rathore/Documents/Logical_Rhythmm/Q3/board_states_test'

process_chessboards_separately(test_folder, piece_dir, output_test_folder)

In [ ]:
train_folder = '/Users/rathore/Documents/Logical_Rhythmm/Q3/train'
output_train_folder = '/Users/rathore/Documents/Logical_Rhythmm/Q3/board_states_train'

process_chessboards_separately(train_folder, piece_dir, output_train_folder)

In [ ]:
train_df = pd.read_csv('/Users/rathore/Documents/Logical_Rhythmm/Q3/train.csv') 
board_states_dir = '/Users/rathore/Documents/Logical_Rhythmm/Q3/board_states'
train_csv_path = '/Users/rathore/Documents/Logical_Rhythmm/Q3/train.csv'

train_df = pd.read_csv(train_csv_path)
train_ids = set(train_df['id'].astype(str))  # Convert IDs to string for consistency

board_files = os.listdir(board_states_dir)
board_ids = set(filename.split('_')[0] for filename in board_files if filename.endswith('.csv'))

missing_in_train = board_ids - train_ids
missing_in_board_states = train_ids - board_ids


print("IDs missing in > train.csv:", missing_in_train)
print("IDs missing in > board_states directory:", missing_in_board_states)

In [ ]:
train_csv_path = '/Users/rathore/Documents/Logical_Rhythmm/Q3/train.csv'
train_df = pd.read_csv(train_csv_path)
train_df = train_df[train_df['id'] != 11161]
train_df.to_csv(train_csv_path, index=False)